<a href="https://colab.research.google.com/github/faisalnawazmir/Econometrics-ML_for_Finance/blob/data/Copy_of_Fama_MacBeth_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm 
from tabulate import tabulate
import time
np.random.seed(12345)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


$Y$: Stock return data (T x N matrix)
By CAPM,

$$r_{i,t} = r_f + \beta_i \cdot mkt_t + \epsilon_{i,t}.$$

$$Y = r_f + \vec{\beta}^T \vec{mkt} + \Omega.$$


In [3]:
rf = 0.01 # risk free rate
N = 2000  # of firms in an economy
T = 2520  # length of periods 
dt = 1/252. # length of a time unit (annualized)
s = 0.5*dt**.05 # average idiosyncratic risks 

# to specify the risk of a firm
beta_true = np.random.random((N,1)) # true betas of N firms
mkt = 0.06*dt + np.random.standard_normal((T,1)) # market factor
# to simulate stock return data upon capital asset pricing model (CAPM)
data = rf*dt + np.transpose(beta_true) * mkt + s*np.random.standard_normal((T,N))
data.shape

(2520, 2000)

In [4]:
(np.transpose(beta_true) * mkt).shape

(2520, 2000)

We regard data given and estimate the model with Fama-MacBeth regression.

In [5]:
# Test
y = data[:,12] # pick a firm's time series data
model = LinearRegression()
model.fit(mkt,y)
print('Intercept:', model.intercept_)
print('Estimated Coefficient:', model.coef_[0])
print('True Coefficient:', beta_true[12][0])
print('Coefficient of determination (R-squared) %.2f' 
      % r2_score(y, model.predict(mkt)), '\n')


# Alternatively
x = sm.add_constant(mkt)
model = sm.OLS(y,x).fit()
print(model.summary())

# Linear algebra 
x = np.c_[np.ones(T),mkt]
M = np.linalg.inv(x.T @ x) @ x.T 
print(M @ y)

Intercept: 0.0034267901570485433
Estimated Coefficient: 0.5952905170613728
True Coefficient: 0.6036431486612096
Coefficient of determination (R-squared) 0.73 

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     6761.
Date:                Thu, 08 Apr 2021   Prob (F-statistic):               0.00
Time:                        05:07:37   Log-Likelihood:                -1062.6
No. Observations:                2520   AIC:                             2129.
Df Residuals:                    2518   BIC:                             2141.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P

To explain the linear algebra in our model 

$$ X \equiv \begin{bmatrix} 1,x_1 \\1, x_2 \\ ... \\1, x_T \end{bmatrix}$$
$$ \hat{\beta} = (X'X)^{-1}X'y \equiv My$$

To estimate all betas simultaneously, we can run:

$$ \hat{B_{2xN}} = (X'X)^{-1}X' \cdot Y_{TxN} \equiv M \cdot Y_{TxN}$$



In [6]:
# time series regression for all N firms (FM step 1)
# regress time series stock return data of each firm one-by-one to multifactors
# here we have only one factor for simplicity 

b = np.array([]) # save market betas here 
for i in range(N): # time series regression for N firms one-by-one to esimate their betas
  model = LinearRegression()
  model.fit(mkt,data[:,i])
  b = np.r_[b, model.coef_]

# To check estimated b with linear algebra 
bhat = (M @ data).T[:,1]
bhat.shape

(2000,)

In [ ]:
print(tabulate(list(zip(beta_true,b,bhat))))# it is very large output so I clear output

In [8]:
# cross-sectional regression for all T times (FM step 2)
# regres cross section of stock return on each firm's beta (time-by-time)
b = np.reshape(b,(N,1))
f = np.array([]) # store estiamted mkt factor returns here
for i in range(T): # FM step 2: time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data[i,:]) # cross section regression part!
  f = np.r_[f,model.coef_]

df = pd.DataFrame(f)
print(df.describe())
print('average factor return:', df.mean())
print('standard error of factor return (without Shanken correction):', df.std()/T**.5)



                 0
count  2520.000000
mean      0.014260
std       1.016108
min      -3.331924
25%      -0.658718
50%      -0.000033
75%       0.685279
max       3.580075
average factor return: 0    0.01426
dtype: float64
standard error of factor return (without Shanken correction): 0    0.020241
dtype: float64


In [9]:
# Let us practice FM regression one more time with multifactor model
# Here we will illustrate Fama-French three factor model.
# Three factors are market (mkt), size (smb), and value (hml) factors

# simulate factor loadings (ie, coefficients)
b_mkt = np.random.random((1,N)) # true loadings for market factor
b_smb = np.random.random((1,N)) # true loadings for size factor
b_hml = np.random.random((1,N)) # true loadings for value factor

# simulate factor returns
mkt = 0.06*dt + np.random.standard_normal((T,1)) # market factor
smb = 0.06*dt + np.random.standard_normal((T,1)) # size factor
hml = 0.06*dt + np.random.standard_normal((T,1)) # value factor

# simulate data
data = rf*dt + b_mkt*mkt + b_smb*smb + b_hml*hml + s*np.random.standard_normal((T,N))
data.shape

(2520, 2000)

In [10]:
# Step 1: time series regression firm by firm
fctrs = np.c_[mkt, smb, hml]
b = np.zeros(3)
for i in range(N): # firm-by-firm time series regression
  model = LinearRegression()
  model.fit(fctrs,data[:,i])
  b = np.c_[b,model.coef_]

b  = np.transpose(b[:,1:])
b.shape

(2000, 3)

In [11]:
# Step 2: cross sectional regression time-by-time
f = np.zeros(3)
for i in range(T): # time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data[i,:])
  f = np.c_[f,model.coef_]

f = np.transpose(f[:,1:])
df = pd.DataFrame(f)
df.columns = ['mkt','smb','hml']
print(df.describe())
print('Median estimate:', np.median(f,axis=0))
print('Standard errors (without Shanken correction):', np.std(f,axis=0)/T**.5)

               mkt          smb          hml
count  2520.000000  2520.000000  2520.000000
mean     -0.003421     0.011328     0.000661
std       0.995024     0.983638     0.993616
min      -3.479207    -3.922783    -3.245370
25%      -0.687729    -0.681748    -0.682121
50%      -0.013429     0.022652    -0.016188
75%       0.677624     0.680134     0.659688
max       3.499484     3.324647     3.169219
Median estimate: [-0.01342926  0.02265249 -0.0161883 ]
Standard errors (without Shanken correction): [0.01981743 0.01959064 0.01978938]


In [ ]:
# Check the answers with linear algebra 
# Step 1: Time series regression
x = np.c_[np.ones(T),fctrs]
M = np.linalg.inv(x.T @ x) @ x.T
bhat = M @ data
bhat = bhat[1:,:].T

# Step 2: cross sectional regression
x = np.c_[np.ones(N),bhat]
M = np.linalg.inv(x.T @ x) @ x.T
fhat = (M @ data.T)[1:,:].T
df = pd.DataFrame(fhat)
df.columns = ['mkt','smb','hml']
print(df.describe())


               mkt          smb          hml
count  2520.000000  2520.000000  2520.000000
mean     -0.008183    -0.024769    -0.012406
std       0.993373     0.989079     0.996948
min      -3.663301    -2.944357    -3.190319
25%      -0.677554    -0.711218    -0.687829
50%      -0.001417    -0.034952    -0.002823
75%       0.685244     0.649408     0.637199
max       3.233734     2.930666     3.915250


In [12]:
# Compute standard error with bootstrapping (Shanken correction unnecessary)
# One simple bootstrapping 

indices = np.random.randint(low=0,high=T, size=T)
new_data = data[indices,:]
new_fctrs = fctrs[indices,:]
xx = indices[0]
print(xx)
print(new_data[0,:])
print(data[xx,:])

1603
[-0.08606548  0.61997345  1.54069753 ...  1.01707778 -0.36269035
  0.17821955]
[-0.08606548  0.61997345  1.54069753 ...  1.01707778 -0.36269035
  0.17821955]


In [13]:
# Check the answers with linear algebra 
# Step 1: Time series regression
x = np.c_[np.ones(T),fctrs]
M = np.linalg.inv(x.T @ x) @ x.T
bhat = (M @ data)[1:,:].T

# Step 2: cross sectional regression
x = np.c_[np.ones(N),bhat]
M = np.linalg.inv(x.T @ x) @ x.T
fhat = (M @ data.T)[1:,:].T
df = pd.DataFrame(fhat)
df.columns = ['mkt','smb','hml']
E_fhat = np.average(fhat,axis=0)
print(E_fhat)

def efhats(data,fctrs): # function to compute average estimated factor return
  T = data.shape[0]
  N = data.shape[1]
  # Step 1: Time series regression
  x = np.c_[np.ones(T),fctrs]
  M = np.linalg.inv(x.T @ x) @ x.T
  bhat = (M @ data)[1:,:].T

  # Step 2: cross sectional regression
  # Include other control variables here (eg, firm characteristics)
  x = np.c_[np.ones(N),bhat]
  M = np.linalg.inv(x.T @ x) @ x.T
  fhat = (M @ data.T)[1:,:].T
  df = pd.DataFrame(fhat)
  df.columns = ['mkt','smb','hml']
  E_fhat = np.average(fhat,axis=0)
  return E_fhat 

print(efhats(data,fctrs))

[-0.00342085  0.01132761  0.00066094]
[-0.00342085  0.01132761  0.00066094]


In [14]:
E_fhat = efhats(data,fctrs)
simn = 10**3 # of bootstrapping
t = time.time() # bootstrapping start time 
for i in range(simn): 
  indices = np.random.randint(low=0,high=T,size=T) # resample by rows
  new_data = data[indices,:] # resampled stock return data
  new_fctrs = fctrs[indices,:] # resampled factor return data
  E_fhat = np.c_[E_fhat,efhats(new_data,new_fctrs)]
  # to check bootstrapping status
  if i % 100 == 0: print(i)

print('Simulation duration:', time.time()-t, 'seconds.')
df = pd.DataFrame(E_fhat.T)
df.columns = ['mkt','smb','hml']
print(df.describe())

0
100
200
300
400
500
600
700
800
900
Simulation duration: 40.533711671829224 seconds.
               mkt          smb          hml
count  1001.000000  1001.000000  1001.000000
mean     -0.003440     0.011409     0.000989
std       0.019854     0.019726     0.020119
min      -0.081165    -0.056728    -0.052518
25%      -0.016360    -0.001080    -0.012478
50%      -0.003115     0.011341     0.001272
75%       0.009721     0.023982     0.014948
max       0.068161     0.065659     0.068643


In [15]:
print('FM regression esimate of expeced factor returns: \n', np.average(E_fhat.T,axis=0))
print('FM regression esimate of standard errors: \n', np.std(E_fhat.T,axis=0))

FM regression esimate of expeced factor returns: 
 [-0.00344017  0.01140894  0.00098873]
FM regression esimate of standard errors: 
 [0.01984397 0.01971612 0.02010855]
